In [2]:
import tensorflow as tf

# Load the data
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    '/Users/sathvikchowdaryveerapaneni/Desktop/GitHub/Project/IOS_APP/FER/Data/train',
    validation_split=0.2,
    subset='training',
    seed=123,
    labels='inferred',
    label_mode='categorical',
    image_size=(48, 48),
    color_mode='grayscale',
    batch_size=32
)

val_data = tf.keras.preprocessing.image_dataset_from_directory(
    '/Users/sathvikchowdaryveerapaneni/Desktop/GitHub/Project/IOS_APP/FER/Data/train',
    validation_split=0.2,
    subset='validation',
    seed=123,
    labels='inferred',
    label_mode='categorical',
    image_size=(48, 48),
    color_mode='grayscale',
    batch_size=32
)

# Normalize the data
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
train_data = train_data.map(lambda x, y: (normalization_layer(x), y))
val_data = val_data.map(lambda x, y: (normalization_layer(x), y))


Found 28709 files belonging to 7 classes.
Using 22968 files for training.
Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Found 28709 files belonging to 7 classes.
Using 5741 files for validation.
